In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Nov 22 20:18:19 2021

@author: xuquanfeng
"""
from PIL import Image
import torch
from torchvision import datasets,transforms,utils,models
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import random
import os
import datetime
import torchvision
import torch.nn.functional as F
from astropy.io import fits
import copy 
import matplotlib.pyplot as plt
from torch import optim
#设置随机种子
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.enabled = True
    os.environ['PYTHONHASHSEED'] = str(seed)
setup_seed(10)
# Hyper parameters
if not os.path.exists('./model'):
    os.mkdir('./model')
if not os.path.exists('./train_proces'):
    os.mkdir('./train_proces')
num_epochs = 20   #循环次数
batch_size = 128    #每次投喂数据量
learning_rate = 0.00001   #学习率
num_var = 40
momentum = 0.8
k = 1

class MyDataset(torch.utils.data.Dataset): 
    def __init__(self, datatxt, transform=None, target_transform=None):
        super(MyDataset, self).__init__()
        fh = open(datatxt, 'r')
        imgs = []
        for line in fh:
            words = line.rstrip().split()
            imgs.append((words[0],words[1]))
        imgf = [i[0] for i in imgs]
        imgd = [i[1] for i in imgs]
        aa = [i for i in range(len(imgf))]
        random.shuffle(aa)
        imgg = [imgd[i] for i in aa]
        imgs = [[imgf[i], imgg[i]] for i in range(len(imgf))]
        self.imgs = imgs
        self.aa = aa
        self.transform = transform
        self.target_transform = target_transform

    def __getitem__(self, index):  # 这个方法是必须要有的，用于按照索引读取每个元素的具体内容
        fn = self.imgs[index]  # fn是图片path #fn和label分别获得imgs[index]也即是刚才每行中word[0]和word[1]的信息
        hdu = fits.open(fn[0])
        img = hdu[0].data
        img = np.array(img,dtype=np.float32)
        hdu.close()
        hdu = fits.open(fn[1])
        oimg = hdu[0].data
        oimg = np.array(oimg,dtype=np.float32)
        hdu.close()
        if self.transform is not None:
            img = self.transform(img)
            img = img.permute(1,0,2)
            oimg = self.transform(oimg)
            oimg = oimg.permute(1,0,2)
        return img,oimg,fn
    def __len__(self):  # 这个函数也必须要写，它返回的是数据集的长度，也就是多少张图片，要和loader的长度作区分
        return len(self.imgs)


print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

class VAE(nn.Module):
    def __init__(self,num_var):
        super(VAE, self).__init__()
        modules = []
        hidden_dims = [32, 64, 128]
        self.hidden_dims = hidden_dims
        in_channels = 3
        latent_dim = num_var
        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    # nn.BatchNorm2d(h_dim),
                    nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2),return_indices=True),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*16, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*16, latent_dim)
        
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * 16)

        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.MaxUnpool2d((2, 2), stride=(2, 2)),
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    # nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )

        self.decoder = nn.Sequential(*modules)
        self.final_layer = nn.Sequential(
                            nn.MaxUnpool2d((2, 2), stride=(2, 2)),
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            # nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= 3,
                                      kernel_size= 3, padding= 1),
                            nn.ReLU())

    def encode(self, x):
        result = x
        idx = []
        for i in range(len(self.hidden_dims)):
            result,indices = self.encoder[i][:2](result)
            idx.append(indices)
            result = self.encoder[i][2](result)        
        self.idx = idx
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)
        return mu, log_var

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.FloatTensor(std.size()).normal_().to(device)
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        result = self.decoder_input(z)
        result = result.view(len(result), 128, 4, 4)
        for i in range(len(self.hidden_dims)-1):
            result = self.decoder[i][0](result,self.idx[len(self.hidden_dims)-1-i])
            result = self.decoder[i][1:](result)
        # result = self.decoder(result)
        result = self.final_layer[0](result,self.idx[0])
        result = self.final_layer[1:](result)
        return result

    def forward(self, x):
        # x = x.view(len(x),-1)
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar
torch.cuda.empty_cache()
model = torch.load('/data/xqf/VAE2/model/vae_40_best_1.pth')

# print(model)
# Device configuration  判断能否使用cuda加速
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model1 = copy.deepcopy(model)

reconstruction_function = nn.MSELoss(size_average=True)

def loss_function1(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + k*KLD


PyTorch Version:  1.10.0+cu113
Torchvision Version:  0.11.1+cu113


/data/xqf/anaconda3/envs/xu/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [14]:
class MMDLoss(nn.Module):
    '''
    计算源域数据和目标域数据的MMD距离
    Params:
    source: 源域数据（n * len(x))
    target: 目标域数据（m * len(y))
    kernel_mul:
    kernel_num: 取不同高斯核的数量
    fix_sigma: 不同高斯核的sigma值
    Return:
    loss: MMD loss
    '''
    def __init__(self, kernel_type='rbf', kernel_mul=2.0, kernel_num=5, fix_sigma=None, **kwargs):
        super(MMDLoss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None
        self.kernel_type = kernel_type

    def guassian_kernel(self, source, target, kernel_mul, kernel_num, fix_sigma):
        n_samples = int(source.size()[0]) + int(target.size()[0])
        total = torch.cat([source, target], dim=0)
        total0 = total.unsqueeze(0).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        total1 = total.unsqueeze(1).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))
        L2_distance = ((total0-total1)**2).sum(2)
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul**i)
                          for i in range(kernel_num)]
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp)
                      for bandwidth_temp in bandwidth_list]
        return sum(kernel_val)

    def linear_mmd2(self, f_of_X, f_of_Y):
        loss = 0.0
        delta = f_of_X.float().mean(0) - f_of_Y.float().mean(0)
        loss = delta.dot(delta.T)
        return loss

    def forward(self, source, target):
        if self.kernel_type == 'linear':
            return self.linear_mmd2(source, target)
        elif self.kernel_type == 'rbf':
            batch_size = int(source.size()[0])
            kernels = self.guassian_kernel(
                source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
            XX = torch.mean(kernels[:batch_size, :batch_size])
            YY = torch.mean(kernels[batch_size:, batch_size:])
            XY = torch.mean(kernels[:batch_size, batch_size:])
            YX = torch.mean(kernels[batch_size:, :batch_size])
            loss = torch.mean(XX + YY - XY - YX)
            return loss
def loss_function(recon_x, x, mu, logvar,recon_x1, x1, mu1, logvar1):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    MMD = MMDLoss()
    l = loss_function1(recon_x, x, mu, logvar)
    l1 = loss_function1(recon_x1, x1, mu1, logvar1)    
    return (l+l1)/2,MMD(source=mu, target=mu1)

In [15]:
train_loss11 = open('./train_proces/train_'+str(num_var)+'_'+str(k)+'.txt', 'w')
train_data = MyDataset(datatxt='train_tal.txt', transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle=True,num_workers=20)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



strattime = datetime.datetime.now()
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(train_loader):
        img,oimg,_ = data
        img = Variable(img)
        img = img.to(device)
        oimg = Variable(oimg)
        oimg = oimg.to(device)
        optimizer.zero_grad()
        cimg, mu, lov = model(img)
        omu, olov = model1.encode(oimg)
        z = model1.reparametrize(omu, olov)
        # print(z.size())
        ocimg = model1.decode(z)
        loss1,loss2 = loss_function(cimg, img, mu, lov, ocimg, oimg, omu, olov)
        loss2 *= len(img)
        loss = loss1 + 10*loss2
        loss.backward()
        # train_loss += loss.data[0]
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 10 == 0:
            endtime = datetime.datetime.now()
            asd = str('Train Epoch: {} [{}/{} ({:.0f}%)] MSE_Loss: {:.6f}  MMD_Loss: {:.6f}  Loss: {:.6f} time:{:.2f}s'.format(
                epoch,
                batch_idx * len(img),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss1.item() / len(img),
                loss2.item() / len(img),
                loss.item() / len(img),
                (endtime-strattime).seconds))
            print(asd)
            train_loss11.write(asd+'\n')
            # torch.save(model, './model/b_vae'+str(epoch)+'_'+str(batch_idx)+'.pth')
    if epoch == 0:
        best_loss = train_loss / len(train_loader.dataset)
    if epoch > 0 and best_loss > train_loss / len(train_loader.dataset):
        best_loss = train_loss / len(train_loader.dataset)
        asds = 'Save Best Model!'
        print(asds)
        train_loss11.write(asds+'\n')
        torch.save(model, './model/vae_'+str(num_var)+'_'+str(k)+'_best.pth')
    asds = str('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))
    print(asds)
    train_loss11.write(asds+'\n')
train_loss11.close()

Train Epoch: 0 [0/3434 (0%)] MSE_Loss: 95.142593  MMD_Loss: 0.075155  Loss: 95.894142 time:2.00s
Train Epoch: 0 [1280/3434 (37%)] MSE_Loss: 76.561981  MMD_Loss: 0.128728  Loss: 77.849258 time:5.00s
Train Epoch: 0 [2560/3434 (74%)] MSE_Loss: 67.318802  MMD_Loss: 0.106715  Loss: 68.385956 time:7.00s
====> Epoch: 0 Average loss: 78.7040
Train Epoch: 1 [0/3434 (0%)] MSE_Loss: 66.315010  MMD_Loss: 0.149502  Loss: 67.810028 time:11.00s
Train Epoch: 1 [1280/3434 (37%)] MSE_Loss: 62.696274  MMD_Loss: 0.192153  Loss: 64.617798 time:14.00s
Train Epoch: 1 [2560/3434 (74%)] MSE_Loss: 74.064636  MMD_Loss: 0.156412  Loss: 75.628754 time:17.00s
Save Best Model!
====> Epoch: 1 Average loss: 71.4790
Train Epoch: 2 [0/3434 (0%)] MSE_Loss: 63.412537  MMD_Loss: 0.186931  Loss: 65.281853 time:21.00s
Train Epoch: 2 [1280/3434 (37%)] MSE_Loss: 61.869717  MMD_Loss: 0.207717  Loss: 63.946888 time:24.00s
Train Epoch: 2 [2560/3434 (74%)] MSE_Loss: 54.702812  MMD_Loss: 0.264065  Loss: 57.343460 time:27.00s
Save B

In [16]:
def _fspecial_gauss_1d(size, sigma):
    coords = torch.arange(size).to(dtype=torch.float)
    coords -= size//2
    g = torch.exp(-(coords**2) / (2*sigma**2))
    g /= g.sum()
    return g.unsqueeze(0).unsqueeze(0)
    
def gaussian_filter(input, win):
    N, C, H, W = input.shape
    out = F.conv2d(input, win, stride=1, padding=0, groups=C)
    out = F.conv2d(out, win.transpose(2, 3), stride=1, padding=0, groups=C)
    return out

def _ssim(X, Y, win, data_range=1023, size_average=True, full=False):
    K1 = 0.01
    K2 = 0.03
    batch, channel, height, width = X.shape
    compensation = 1.0

    C1 = (K1 * data_range)**2
    C2 = (K2 * data_range)**2

    win = win.to(X.device, dtype=X.dtype)

    mu1 = gaussian_filter(X, win)
    mu2 = gaussian_filter(Y, win)

    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2

    sigma1_sq = compensation * ( gaussian_filter(X * X, win) - mu1_sq )
    sigma2_sq = compensation * ( gaussian_filter(Y * Y, win) - mu2_sq )
    sigma12   = compensation * ( gaussian_filter(X * Y, win) - mu1_mu2 )

    cs_map = (2 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    ssim_map = ((2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1)) * cs_map

    if size_average:
        ssim_val = ssim_map.mean()
        cs = cs_map.mean()
    else:
        ssim_val = ssim_map.mean(-1).mean(-1).mean(-1)  # reduce along CHW
        cs = cs_map.mean(-1).mean(-1).mean(-1)

    if full:
        return ssim_val, cs
    else:
        return ssim_val

def ssim(X, Y, win_size=11, win_sigma=10, win=None, data_range=1, size_average=True, full=False):

    if len(X.shape) != 4:
        raise ValueError('Input images must 4-d tensor.')

    if not X.type() == Y.type():
        raise ValueError('Input images must have the same dtype.')

    if not X.shape == Y.shape:
        raise ValueError('Input images must have the same dimensions.')

    if not (win_size % 2 == 1):
        raise ValueError('Window size must be odd.')

    win_sigma = win_sigma
    if win is None:
        win = _fspecial_gauss_1d(win_size, win_sigma)
        win = win.repeat(X.shape[1], 1, 1, 1)
    else:
        win_size = win.shape[-1]

    ssim_val, cs = _ssim(X, Y,
                         win=win,
                         data_range=data_range,
                         size_average=False,
                         full=True)
    if size_average:
        ssim_val = ssim_val.mean()
        cs = cs.mean()

    if full:
        return ssim_val, cs
    else:
        return ssim_val


    def __init__(self, win_size=11, win_sigma=1.5, data_range=255, size_average=True, channel=3, weights=None):
        super(MS_SSIM, self).__init__()
        self.win = _fspecial_gauss_1d(
            win_size, win_sigma).repeat(channel, 1, 1, 1)
        self.size_average = size_average
        self.data_range = data_range
        self.weights = weights

    def forward(self, X, Y):
        return ms_ssim(X, Y, win=self.win, size_average=self.size_average, data_range=self.data_range, weights=self.weights)


In [27]:
train_data = MyDataset(datatxt='train_tal.txt', transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size = batch_size, shuffle=False,num_workers=20)

if not os.path.exists('./result'):
    os.mkdir('./result')
model.eval()
from tqdm import tqdm
sssi = []
sss2 = []
with torch.no_grad():
    for batch_idx, data in enumerate(tqdm(train_loader)):
        img,oimg,fn = data
        img = Variable(img)
        img = img.to(device)
        oimg = Variable(oimg)
        oimg = oimg.to(device)
        optimizer.zero_grad()
        
        cimg, mu, lov = model(img)
        ocimg, omu, olov = model1(oimg)

        for i in range(len(img)):
            ssim_val = ssim(img[i].unsqueeze(0), cimg[i].unsqueeze(0), data_range=1, size_average=True,)
            qw = [fn[0][i]]
            qw.append(ssim_val.cpu().detach().numpy())
            qw.extend(mu[i].cpu().detach().numpy())
            sssi.append(qw)
            qw = [fn[1][i]]
            # qw.append(fn[1][i])
            ssim_val = ssim(oimg[i].unsqueeze(0), ocimg[i].unsqueeze(0), data_range=1, size_average=True,)
            qw.append(ssim_val.cpu().detach().numpy())
            qw.extend(omu[i].cpu().detach().numpy())
            sss2.append(qw)
    bb = np.argsort(train_data.aa)
    sssii = [sssi[i].extend(sss2[bb[i]]) for i in range(len(sssi))]

    dd = np.array(sssi)
    print(len(dd))
    # np.save(pt+'result_ssim.npy',dd)
    np.save('./result/resu_'+str(num_var)+'_'+str(k)+'_all.npy',dd)

  0%|          | 0/27 [00:00<?, ?it/s]

100%|██████████| 27/27 [00:12<00:00,  2.14it/s]


3434


In [29]:
sssi[0]

['/data/GZ_Decals/MGS_out_DECaLS/175.89149808110213_21.67228737779398_0.262_grz_.fits',
 array(0.6318049, dtype=float32),
 -0.10113665,
 0.49029487,
 0.46241808,
 -0.008596858,
 -0.39838147,
 0.26835242,
 0.18839383,
 -0.4677663,
 -0.9939542,
 0.5922526,
 -0.21539639,
 0.052440077,
 -0.05354116,
 -0.55313927,
 -0.8192193,
 0.6176385,
 0.69971794,
 0.44840464,
 0.35968605,
 -0.868997,
 -0.48039755,
 0.006876337,
 0.35151902,
 -0.4821446,
 0.70668,
 -0.3762942,
 -0.50149524,
 -0.7670771,
 0.06952908,
 -0.3410776,
 0.3420713,
 0.20182034,
 -0.7638455,
 -0.53207695,
 0.33097535,
 0.26282936,
 -0.47867802,
 -0.34963456,
 -0.46141088,
 0.41259444,
 '/data/GZ_Decals/nomerge/175.8905862789053_21.66981935317594_0.fits',
 array(0.8620221, dtype=float32),
 -0.1810664,
 0.3472166,
 0.5991349,
 -0.60958856,
 0.020077812,
 0.5928264,
 0.7723357,
 -0.9057603,
 -0.89147216,
 0.55130893,
 0.3216625,
 -0.087235205,
 -0.18043077,
 -0.8762876,
 -0.4985384,
 1.1762805,
 0.10030317,
 0.079826385,
 0.6553646

In [21]:
print(train_data.aa)

[2343, 2568, 2103, 802, 2523, 686, 2760, 3158, 1512, 3181, 351, 213, 514, 103, 1502, 1252, 2039, 48, 2506, 2045, 1201, 398, 1775, 337, 1858, 1919, 1594, 3383, 1297, 2095, 2236, 3281, 78, 3236, 3180, 1737, 187, 1326, 1877, 2260, 1362, 2839, 2934, 2910, 484, 1708, 1825, 833, 3367, 204, 818, 2352, 2643, 3129, 184, 2615, 3197, 1488, 2481, 614, 2539, 3128, 2945, 1796, 1990, 3109, 185, 417, 631, 451, 143, 3069, 481, 3007, 2412, 2392, 1132, 367, 1261, 2194, 3372, 2957, 261, 1499, 138, 208, 933, 2016, 2242, 349, 2113, 2642, 3083, 3418, 1256, 2730, 684, 1308, 771, 477, 3312, 1263, 3302, 2333, 359, 1838, 3284, 1822, 2503, 2731, 3432, 807, 1987, 1116, 2613, 1562, 3136, 24, 1126, 1042, 1670, 2780, 1455, 805, 560, 950, 2607, 2366, 632, 446, 2694, 2763, 1329, 3042, 886, 237, 1793, 1626, 114, 1034, 583, 1802, 258, 286, 3182, 1403, 1191, 177, 1897, 2542, 553, 1121, 3177, 1088, 2261, 2824, 3389, 250, 561, 1254, 2372, 2379, 2574, 3199, 2844, 3031, 3374, 1032, 3206, 3015, 2025, 942, 2135, 3105, 3232, 148

In [26]:
bb = np.argsort(train_data.aa)
print(bb)
cc = [train_data.aa[i] for i in bb]
print(cc)

[1837 1780 2102 ... 2381  110 3249]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214

In [20]:
qw

['/data/GZ_Decals/MGS_out_DECaLS/209.69722323971683_-2.796119143808119_0.262_grz_.fits',
 array(0.636351, dtype=float32),
 -0.29157475,
 1.1080443,
 0.6969389,
 -0.020610403,
 -0.28826585,
 0.17531879,
 0.00834981,
 -0.23376767,
 0.049501695,
 0.40572107,
 0.29489738,
 0.47389993,
 -0.2963957,
 -0.76762176,
 -0.6041273,
 0.8933403,
 0.5617152,
 -0.23937546,
 0.14162734,
 -0.66076005,
 -0.10308995,
 -0.15852496,
 -0.049899317,
 0.17609647,
 0.7838063,
 0.041951694,
 -0.5465605,
 -0.6807498,
 0.27584875,
 -1.0349991,
 1.097923,
 0.4099423,
 -0.70314544,
 -1.0785923,
 0.024905168,
 -0.8701954,
 0.41669047,
 -0.7397161,
 0.23750192,
 1.048735,
 '/data/GZ_Decals/nomerge/238.0328306883964_27.583784856873187_0.fits',
 array(0.7068555, dtype=float32),
 -0.39576274,
 0.8755446,
 0.6596966,
 -0.16872467,
 -0.6784575,
 0.05666391,
 0.62383527,
 -0.82780844,
 -1.226114,
 1.1190218,
 -0.08839746,
 -0.13593765,
 -0.105748,
 -0.8570283,
 -0.77886534,
 1.0021926,
 0.84907526,
 0.35242316,
 0.6474615,
